# UniProt Analysis

Resources:

- [XML Schema](https://www.uniprot.org/docs/uniprot.xsd)
- [User Manual](https://www.uniprot.org/help/uniprotkb_manual)
- [Text File Format Manual](https://web.expasy.org/docs/userman.html)

Contents:

- [Schema](#schema): Inferred Spark schema 
- [Accession Numbers](#accession_numbers)
- [Organisms](#organisms)
- [Protein Existence](#protein_existence)

In [1]:
import $file.^.sparkinit, sparkinit._
import $file.^.pathinit, pathinit._
import $ivy.`com.github.pathikrit::better-files:3.8.0`
import $ivy.`com.databricks::spark-xml:0.7.0`
import ss.implicits._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
import better.files._

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
19/12/27 19:37:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


import $file.$          , sparkinit._

import $file.$         , pathinit._

import $ivy.$                                         

import $ivy.$                                

import ss.implicits._

import org.apache.spark.sql.functions._

## Partition

Break the raw xml up into parquet partitions (and save the schema):

In [14]:
// Download from https://www.uniprot.org/downloads (Swiss-Prot only for now)
val path = (File(DATA_CACHE_DIR) / "uniprot" / "uniprot_sprot.xml").toString
val df = ss.read
    .format("com.databricks.spark.xml")
    .option("rowTag", "entry")
    .load(path)

fold at InferSchema.scala:115

202 / 202

19/12/27 19:11:21 WARN Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


path: String = "/home/eczech/data/ot/cache/uniprot/uniprot_sprot.xml"
df: org.apache.spark.sql.package.DataFrame = [_created: string, _dataset: string ... 18 more fields]

<h3><a id="schema">Schema</a></h3>

In [55]:
df.printSchema

root
 |-- _created: string (nullable = true)
 |-- _dataset: string (nullable = true)
 |-- _modified: string (nullable = true)
 |-- _version: long (nullable = true)
 |-- _xmlns: string (nullable = true)
 |-- accession: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- comment: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _error: double (nullable = true)
 |    |    |-- _evidence: string (nullable = true)
 |    |    |-- _locationType: string (nullable = true)
 |    |    |-- _mass: double (nullable = true)
 |    |    |-- _method: string (nullable = true)
 |    |    |-- _name: string (nullable = true)
 |    |    |-- _type: string (nullable = true)
 |    |    |-- absorption: struct (nullable = true)
 |    |    |    |-- max: struct (nullable = true)
 |    |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |    |-- _evidence: string (nullable = true)
 |    |    

In [20]:
val path = (File(DATA_CACHE_DIR) / "uniprot" / "uniprot_sprot.schema.txt")
path.overwrite(df.schema.treeString)

path: File = /home/eczech/data/ot/cache/uniprot/uniprot_sprot.schema.txt
res19_1: File = /home/eczech/data/ot/cache/uniprot/uniprot_sprot.schema.txt

In [16]:
val path = (File(DATA_CACHE_DIR) / "uniprot" / "uniprot_sprot.parquet").toString
df.write.format("parquet").save(path)

save at cmd15.sc:2

202 / 202

path: String = "/home/eczech/data/ot/cache/uniprot/uniprot_sprot.parquet"

## Load 

In [3]:
val df = ss.read.parquet((File(DATA_CACHE_DIR) / "uniprot" / "uniprot_sprot.parquet").toString)

parquet at cmd2.sc:1

1 / 1

19/12/27 19:38:47 WARN Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


df: org.apache.spark.sql.package.DataFrame = [_created: string, _dataset: string ... 18 more fields]

In [113]:
// CONSTANTS
val NCBI_HUMAN = 9606

// Set common projections
val PROJ_ENTRY_MAIN = Seq(
    $"accession".getItem(0).as("primary_accession"), 
    $"name".as("entry_name"), 
    $"protein.recommendedName.fullName._VALUE".as("protein_name"))

NCBI_HUMAN: Int = 9606
PROJ_ENTRY_MAIN: Seq[org.apache.spark.sql.Column] = List(
  accession[0] AS `primary_accession`,
  name AS `entry_name`,
  protein.recommendedName.fullName._VALUE AS `protein_name`
)

<h3><a id="accession_numbers">Accession Numbers</a></h3>

See https://www.uniprot.org/help/accession_numbers.  When entries are split or merged, they are assigned new accession numbers so many entries have multiple ids but the first should be used as the primary accession.  

See https://www.uniprot.org/help/entry_name for information on how entry names are assigned.  Notably, orthologous proteins across species are assigned the same "mnemoic code" (i.e. the "UBL1" in "UBL1_YEAST") even if the gene names differ. Otherwise the mnemoic code appears to often be similar to the gene name.

In [92]:
df.select(size($"accession").as("accession_count"))
    .groupBy("accession_count").count.sort($"count".desc)
    .show(5, false)

show at cmd91.sc:3

16 / 16

show at cmd91.sc:3

1 / 1

+---------------+------+
|accession_count|count |
+---------------+------+
|1              |443515|
|2              |65071 |
|3              |25317 |
|4              |13518 |
|5              |5537  |
+---------------+------+
only showing top 5 rows



In [99]:
// Show multi-accession record sample and associated entry/protein names
df
    .filter(size($"accession") > 1)
    .select((PROJ_ENTRY_MAIN :+ $"accession".as("all_accessions")):_*)
    .show(5, false)

show at cmd98.sc:3

1 / 1

+-----------------+-----------+------------------------------------------+------------------------------------------------+
|primary_accession|entry_name |protein_name                              |all_accessions                                  |
+-----------------+-----------+------------------------------------------+------------------------------------------------+
|P35127           |UBL1_YEAST |Ubiquitin carboxyl-terminal hydrolase YUH1|[P35127, D6VWR8]                                |
|O95164           |UBL3_HUMAN |Ubiquitin-like protein 3                  |[O95164, B2R4J1, Q5RL72, Q5VZS0, Q6FIG8, Q96SG7]|
|Q9Z2M6           |UBL3_MOUSE |Ubiquitin-like protein 3                  |[Q9Z2M6, A4FTW0, Q3UKM1]                        |
|P11441           |UBL4A_HUMAN|Ubiquitin-like protein 4A                 |[P11441, Q5HY80]                                |
|Q0D261           |UBL4A_XENLA|Ubiquitin-like protein 4A                 |[Q0D261, A7YT11, Q52KN4]                        |
+-------

<h3><a id="organisms">Organisms</a></h3>

Relevant links:

- https://www.uniprot.org/help/organism-name: general information on organisms
- https://www.uniprot.org/help/taxonomic_identifier: info on use of NCBI taxonomy ids
- https://www.uniprot.org/docs/speclist: full list of supported organisms.

In [27]:
df.select("organism").printSchema

root
 |-- organism: struct (nullable = true)
 |    |-- _evidence: string (nullable = true)
 |    |-- dbReference: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _id: long (nullable = true)
 |    |    |-- _type: string (nullable = true)
 |    |-- lineage: struct (nullable = true)
 |    |    |-- taxon: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |-- name: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |-- _type: string (nullable = true)



In [105]:
// The "id" property in the "dbReference" field is the primary accession for species
df.select("organism.dbReference._type", "organism.dbReference._id").show(3, false)

show at cmd104.sc:1

1 / 1

+-------------+------+
|_type        |_id   |
+-------------+------+
|NCBI Taxonomy|349521|
|NCBI Taxonomy|349124|
|NCBI Taxonomy|572265|
+-------------+------+
only showing top 3 rows



In [108]:
// Show that all ids come from NCBI and that over 13k species are represented
df.groupBy("organism.dbReference._type").agg(countDistinct("organism.dbReference._id")).show

show at cmd107.sc:1

16 / 16

show at cmd107.sc:1

1 / 1

show at cmd107.sc:1

1 / 1

+-------------+----------------------------------------+
|        _type|count(DISTINCT organism.dbReference._id)|
+-------------+----------------------------------------+
|NCBI Taxonomy|                                   13856|
+-------------+----------------------------------------+



In [103]:
// Show distinct counts of organism names by type (note that the "scientific" names match cardinality of NCBI ids above)
df
    .withColumn("organism", explode($"organism.name"))
    .select("organism.*")
    .groupBy("_type").agg(countDistinct("_VALUE"))
    .show(3, false)

show at cmd102.sc:5

16 / 16

show at cmd102.sc:5

1 / 1

show at cmd102.sc:5

1 / 1

+----------+----------------------+
|_type     |count(DISTINCT _VALUE)|
+----------+----------------------+
|scientific|13856                 |
|common    |8094                  |
|synonym   |3422                  |
+----------+----------------------+



In [117]:
// Show that organism name types are unique for each record (e.g. there are never multiple "scientific" names)
df
    .withColumn("organism_name", explode($"organism.name"))
    .groupBy($"accession".getItem(0), $"organism_name._type")
    .count.groupBy("count").count.show

show at cmd116.sc:4

16 / 16

show at cmd116.sc:4

1 / 1

show at cmd116.sc:4

1 / 1

+-----+------+
|count| count|
+-----+------+
|    1|850764|
+-----+------+



In [116]:
// Show organism name for humans
df.filter($"organism.dbReference._id" === NCBI_HUMAN).groupBy($"organism.name").count.show(false)

show at cmd115.sc:1

16 / 16

show at cmd115.sc:1

1 / 1

+---------------------------------------------+-----+
|name                                         |count|
+---------------------------------------------+-----+
|[[Homo sapiens, scientific], [Human, common]]|20367|
+---------------------------------------------+-----+



In [35]:
// Show frequencies of organisms by name
df
    .withColumn("organism", explode($"organism.name")) 
    .select("organism.*")
    // Filter to scientific name will align resulting frequency with counts across entries 
    // rather than counts across multiple records per entry (there is only one scientific name per entry)
    .filter($"_type" === "scientific")
    .groupBy("_VALUE")
    .count
    .sort($"count".desc)
    .show(10, false)

show at cmd34.sc:8

16 / 16

show at cmd34.sc:8

1 / 1

+------------------------------------------------------+-----+
|_VALUE                                                |count|
+------------------------------------------------------+-----+
|Homo sapiens                                          |20367|
|Mus musculus                                          |17027|
|Arabidopsis thaliana                                  |15922|
|Rattus norvegicus                                     |8085 |
|Saccharomyces cerevisiae (strain ATCC 204508 / S288c) |6721 |
|Bos taurus                                            |6008 |
|Schizosaccharomyces pombe (strain 972 / ATCC 24843)   |5140 |
|Escherichia coli (strain K12)                         |4518 |
|Bacillus subtilis (strain 168)                        |4188 |
|Dictyostelium discoideum                              |4149 |
|Caenorhabditis elegans                                |4105 |
|Oryza sativa subsp. japonica                          |4070 |
|Drosophila melanogaster                               

In [119]:
// Show count for exact match on scientific name
df.filter(array_contains($"organism.name", struct(
    lit("Homo sapiens").as("_VALUE"),
    lit("scientific").as("_type")
))).count

count at cmd118.sc:3

16 / 16

count at cmd118.sc:3

1 / 1

res118: Long = 20367L

In [120]:
// Use NCBI id for filter on downstream data and compare count to above
val dfh = df.filter($"organism.dbReference._id" === NCBI_HUMAN)
dfh.count

count at cmd119.sc:2

16 / 16

count at cmd119.sc:2

1 / 1

dfh: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [_created: string, _dataset: string ... 18 more fields]
res119_1: Long = 20367L

<h3><a id="protein_existence">Protein Existence</a></h3>

While most of the UP entries are proteins observed at some point in a laboratory setting, a significant portion of them are also assumed to exist based on transcript measurement or some other less direct method (see https://web.expasy.org/docs/userman.html#PE_line).

In [9]:
dfh.groupBy("proteinExistence._type").count.show(false)

show at cmd8.sc:1

16 / 16

show at cmd8.sc:1

1 / 1

+----------------------------+-----+
|_type                       |count|
+----------------------------+-----+
|evidence at protein level   |15428|
|inferred from homology      |814  |
|evidence at transcript level|3373 |
|uncertain                   |592  |
|predicted                   |160  |
+----------------------------+-----+



### Evidence Codes

Evidence codes are associated with a variety of the different types of evidence provided (see https://web.expasy.org/docs/userman.html#ev_description) and they are also stated at the root level of the entries as a generalization of what curation process an entry underwent.  The codes at this level are:

- ECO:0000269 - [experimental evidence used in manual assertion](https://www.ebi.ac.uk/QuickGO/term/ECO:0000269)
- ECO:0000244 - [combinatorial evidence used in manual assertion](https://www.ebi.ac.uk/QuickGO/term/ECO:0000244)
- ECO:0000255 - [match to sequence model evidence used in manual assertion](https://www.ebi.ac.uk/QuickGO/term/ECO:0000255)
- ECO:0000303 - [author statement without traceable support used in manual assertion](https://www.ebi.ac.uk/QuickGO/term/ECO:0000303)
- ECO:0000305 - [curator inference used in manual assertion](https://www.ebi.ac.uk/QuickGO/term/ECO:0000305)
- ECO:0000250 - [sequence similarity evidence used in manual assertion](https://www.ebi.ac.uk/QuickGO/term/ECO:0000250)
- ECO:0000312 - [imported information used in manual assertion](https://www.ebi.ac.uk/QuickGO/term/ECO:0000312)

In [51]:
// Code frequency, independent of combinations
dfh.withColumn("evidence_codes", explode($"evidence"))
    .groupBy($"evidence_codes._type".as("code")).count
    .sort($"count".desc).show

show at cmd50.sc:3

16 / 16

show at cmd50.sc:3

1 / 1

+-----------+------+
|       code| count|
+-----------+------+
|ECO:0000269|101169|
|ECO:0000244| 43423|
|ECO:0000255| 24583|
|ECO:0000303| 23939|
|ECO:0000305| 22751|
|ECO:0000250| 18315|
|ECO:0000312|  4279|
+-----------+------+



In [54]:
// Most common combinations associated with an individual entry
dfh.select($"accession".getItem(0).as("id"), explode($"evidence").as("evidence"))
    .groupBy("id").agg(collect_set($"evidence._type").as("codes"))
    .groupBy("codes").count
    .sort($"count".desc)
    .show(10, false)

show at cmd53.sc:5

16 / 16

show at cmd53.sc:5

1 / 1

show at cmd53.sc:5

1 / 1

+-------------------------------------------------------------------------------------------+-----+
|codes                                                                                      |count|
+-------------------------------------------------------------------------------------------+-----+
|[ECO:0000305, ECO:0000255, ECO:0000303, ECO:0000244, ECO:0000250, ECO:0000269]             |2741 |
|[ECO:0000305, ECO:0000255, ECO:0000303, ECO:0000250, ECO:0000269]                          |1376 |
|[ECO:0000305, ECO:0000255, ECO:0000244, ECO:0000250, ECO:0000269]                          |1246 |
|[ECO:0000305, ECO:0000255, ECO:0000250, ECO:0000269]                                       |1116 |
|[ECO:0000305, ECO:0000255, ECO:0000303, ECO:0000244, ECO:0000269]                          |1029 |
|[ECO:0000305, ECO:0000303, ECO:0000244, ECO:0000250, ECO:0000269]                          |839  |
|[ECO:0000305, ECO:0000255, ECO:0000303, ECO:0000269]                                       |837  |


### CD Antigen Names

See https://www.uniprot.org/docs/cdlist for a full list of the CD molecules in UniProt.

CD names are typically unique however there are a few cases where the same name is assigned to both receptors and ligands:

In [23]:
dfh.groupBy("protein.cdAntigenName._VALUE")
    .count.sort($"count".desc).show(8, false)

show at cmd22.sc:2

16 / 16

show at cmd22.sc:2

1 / 1

+------+-----+
|_VALUE|count|
+------+-----+
|null  |19969|
|CD36  |3    |
|CD32  |3    |
|CD99  |2    |
|CD87  |1    |
|CD109 |1    |
|CD98  |1    |
|CD37  |1    |
+------+-----+
only showing top 8 rows



In [100]:
// Set common projections
dfh.filter($"protein.cdAntigenName._VALUE" === "CD36")
    .select((PROJ_ENTRY_MAIN :+ flatten($"gene.name")("_VALUE")): _*)
    .show(false)

show at cmd99.sc:3

1 / 1

show at cmd99.sc:3

4 / 4

show at cmd99.sc:3

11 / 11

+-----------------+-----------+-----------------------------------+---------------------------------------+
|primary_accession|entry_name |protein_name                       |flatten(gene.name AS name#27355)._VALUE|
+-----------------+-----------+-----------------------------------+---------------------------------------+
|P16671           |CD36_HUMAN |Platelet glycoprotein 4            |[CD36, GP3B, GP4]                      |
|Q8WTV0           |SCRB1_HUMAN|Scavenger receptor class B member 1|[SCARB1, CD36L1, CLA1]                 |
|Q14108           |SCRB2_HUMAN|Lysosome membrane protein 2        |[SCARB2, CD36L2, LIMP2, LIMPII]        |
+-----------------+-----------+-----------------------------------+---------------------------------------+



### Gene Name Types

See https://www.uniprot.org/help/gene_name (as well as https://web.expasy.org/docs/userman.html#GN_line) for a description of the four types of gene names that can be associated with an entry.  Of note, the official names and synonyms do not appear to be from one source or nomenclature committee but are instead aggregated and maintained by UniProt.  Also, "OrderedLocusName" does not appear in the data while "ORF" does and indicates preliminary gene designations (before they are adequately characterized).  See below for a few human proteins with no official gene name, just an ORF from whatever sequencing project discovered them:

In [66]:
// Show the frequency of distinct gene types that occur for each entry
dfh.select(array_distinct(flatten($"gene.name")("_type")).as("type"))
    .groupBy("type").count.show(false)

show at cmd65.sc:1

16 / 16

show at cmd65.sc:1

1 / 1

+-----------------------+-----+
|type                   |count|
+-----------------------+-----+
|[primary, synonym]     |11594|
|[primary]              |6425 |
|[primary, synonym, ORF]|1502 |
|[primary, ORF]         |677  |
|null                   |146  |
|[ORF]                  |23   |
+-----------------------+-----+



In [101]:
// Pull the records for the examples above that have only an ORF designation
dfh.filter(array_distinct(flatten($"gene.name")("_type")) === array(lit("ORF")))
    .select(PROJ_ENTRY_MAIN ++ Seq(flatten($"gene.name").as("gene"), $"proteinExistence._type".as("protein_existence")):_*)
    .sort($"protein_existence".desc)
    .show(23, false)

show at cmd100.sc:4

16 / 16

+-----------------+-----------+----------------------------------------------------------+-----------------------------------+----------------------------+
|primary_accession|entry_name |protein_name                                              |gene                               |protein_existence           |
+-----------------+-----------+----------------------------------------------------------+-----------------------------------+----------------------------+
|Q9UI72           |YE014_HUMAN|Putative uncharacterized protein PRO0255                  |[[PRO0255,, ORF]]                  |uncertain                   |
|Q9P1D8           |YP008_HUMAN|Putative uncharacterized protein PRO2289                  |[[PRO2289,, ORF]]                  |uncertain                   |
|Q9UHU1           |YK039_HUMAN|Putative uncharacterized protein PRO1716                  |[[PRO1716,, ORF]]                  |uncertain                   |
|Q6UXP9           |YO001_HUMAN|Putative uncharacterized protein 

In [83]:
// Show a few examples of all types of gene names for each type
dfh.select(explode($"gene.name").as("gene"))
    .select(explode($"gene").as("gene"))
    .select("gene.*")
    // Get 10 example values for each gene name type
    .withColumn("rid", row_number.over(Window.partitionBy("_type").orderBy($"_evidence".desc)))
    .filter($"rid" <= 10)
    // Pivot to show examples concatenated horizontally
    .groupBy("rid").pivot("_type").agg(max("_VALUE"))
    .drop("rid").show(10, false)

pivot at cmd82.sc:10

16 / 16

pivot at cmd82.sc:10

1 / 1

pivot at cmd82.sc:10

...

1 / 1

show at cmd82.sc:11

1 / 1

+----------------+-------+--------+
|ORF             |primary|synonym |
+----------------+-------+--------+
|UNQ150/PRO176   |GBA    |NCX1    |
|UNQ2423/PRO4981 |HLA-B  |NGRH1   |
|HSD13           |CYP3A7 |DECTIN2 |
|UNQ403/PRO740   |RAET1E |TWA1    |
|UNQ743/PRO1471  |CYP2J2 |ILT7    |
|PTD019          |EXTL3  |MHF2    |
|CGI-143         |CYP26A1|UTP11L  |
|CDA03           |MRM3   |A3GALT2P|
|UNQ9427/PRO34683|IGHA2  |AAT1    |
|UNQ1850/PRO3580 |SFXN4  |KIAA1731|
+----------------+-------+--------+



import org.apache.spark.sql.expressions.Window



## Comments

In [12]:
dfh.withColumn("comments", explode($"comment")).select("comments.*")
    .printSchema

root
 |-- _VALUE: string (nullable = true)
 |-- _error: double (nullable = true)
 |-- _evidence: string (nullable = true)
 |-- _locationType: string (nullable = true)
 |-- _mass: double (nullable = true)
 |-- _method: string (nullable = true)
 |-- _name: string (nullable = true)
 |-- _type: string (nullable = true)
 |-- absorption: struct (nullable = true)
 |    |-- max: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _evidence: string (nullable = true)
 |    |-- text: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _evidence: string (nullable = true)
 |-- cofactor: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _evidence: string (nullable = true)
 |    |    |-- dbReference: struct (nullable = true)
 |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |-- _id: string (nullable = true)
 |    |    |    |-- _type: string (nullable = true)
 |    |    |-- n

In [18]:
dfh.withColumn("comments", explode($"comment"))
    .groupBy("comments._type")
    .count.sort($"count".desc)
    .show(false)

show at cmd17.sc:4

16 / 16

show at cmd17.sc:4

1 / 1

+-----------------------------+-----+
|_type                        |count|
+-----------------------------+-----+
|interaction                  |55521|
|subcellular location         |17911|
|function                     |17451|
|similarity                   |14346|
|subunit                      |12499|
|sequence caution             |11883|
|alternative products         |10628|
|tissue specificity           |9826 |
|PTM                          |7273 |
|disease                      |6740 |
|catalytic activity           |6545 |
|domain                       |4376 |
|online information           |4165 |
|miscellaneous                |4024 |
|caution                      |2198 |
|induction                    |1885 |
|cofactor                     |1712 |
|pathway                      |1344 |
|activity regulation          |1282 |
|biophysicochemical properties|859  |
+-----------------------------+-----+
only showing top 20 rows

